In [65]:
import pandas as pd

# Refining

In [66]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
EFF_H2_SMR = 159.6 #MJ/kgH2

In [67]:
lcoh_res_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx')
res_be_df = pd.DataFrame(columns=['RES', 'Breakeven price ($/MMBtu)'])
res_be_df['RES'] = lcoh_res_df['RES']
res_be_df['Breakeven price ($/MMBtu)'] = lcoh_res_df.apply(lambda x:x['LCOH']/(EFF_H2_SMR*CONV_MJ_TO_MMBTU), axis=1)
res_be_df

,RES,Breakeven price ($/MMBtu)
0,Grid PEM,34.837997
1,Wind subsidized,37.250875
2,Wind unsubsidized,43.861501
3,Solar PEC,49.579692
4,Solar PV-E,50.901817
5,SMR 89% CCUS,12.758507


In [68]:
res_be_df.to_csv('./results/res_be_refining.csv', header=True, index=False)


## Steel

In [69]:
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
COAL_CONS_RATE = 0.663 #ton-coal/ton-steel for conventional BF/BOF plant
iron_ore_cost = 100 #$/t_ironore
bfbof_iron_cons = 1.226 #t_ironore/t_steel
om_bfbof = 353.25 #$/t_steel
DRICO2Intensity = 40 # kgCO2/ton-DRI`````````````````
ShaftFCAPEX = 250 # $/tDRI/year
EAFCAPEX = 160 # $/tsteel/year
EAFOM  = 24.89 # $/tsteel (EAF and casting)
Steel = 800 # $/tsteel
RatioSteelDRI = 0.9311 # tsteel/tDRI
RatioIronOreDRI = 1.391 # tironore/tDRI


In [70]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [71]:
steel_df = pd.read_excel('h2_demand_bfbof_steel_us_2022.xlsx', sheet_name='processed')
steel_ids = list(steel_df['Plant'])

In [72]:
res_be_df= pd.DataFrame(columns=['Plant', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['Plant'] = steel_df['Plant']
res_be_df.set_index('Plant', inplace=True)


In [73]:
from opt_deployment_steel import get_steel_plant_demand
def compute_res_be(lcoh, lcoe, plant):
  steel_cap_ton_per_annum, h2_dem_kg_per_day, elec_dem_MWe = get_steel_plant_demand(plant)
  costs_dri_eaf = steel_cap_ton_per_annum*(EAFCAPEX + ShaftFCAPEX/RatioSteelDRI + EAFOM +\
            iron_ore_cost*RatioIronOreDRI/RatioSteelDRI)
  costs_h2 =  lcoh*h2_dem_kg_per_day*365
  costs_electricity = lcoe*elec_dem_MWe*24*365
  costs_bfbof = iron_ore_cost*bfbof_iron_cons*steel_cap_ton_per_annum + om_bfbof*steel_cap_ton_per_annum
  coal_ann_consumption = steel_cap_ton_per_annum*COAL_CONS_RATE
  res_be = (costs_h2+costs_electricity+costs_dri_eaf - costs_bfbof)/coal_ann_consumption
  return res_be

In [74]:
for plant in steel_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be(lcoh, lcoe, plant)
    res_be_df.loc[plant, res] = res_be

res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/tcoal)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_steel.csv', header=True, index=False)

# Heat Process

In [75]:

CONV_MWh_to_MJ = 3600 #MJ/MWh
CONV_MJ_TO_MMBTU = 1/1055.05585 #MMBTU/MJ
# GF: Glass Furnace
GFCAPEX = 1340000 #$/MWth
GFLT = 12 # years
WACC = 0.08
#costs = (model.pHeatDem/(CONV_MWh_to_MJ*365))*model.pGFCAPEX*gf_crf#*(1+model.pGFFOM)

In [76]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [77]:
heat_process_df = pd.read_excel('./h2_demand_industry_heat.xlsx', sheet_name='max')
plant_ids = list(heat_process_df['FACILITY_ID'])

In [78]:
res_be_df= pd.DataFrame(columns=['FACILITY_ID', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['FACILITY_ID'] = heat_process_df['FACILITY_ID']
res_be_df.set_index('FACILITY_ID', inplace=True)

In [79]:
from opt_deployment_heat_process import get_plant_demand, get_heat_demand
def compute_res_be_heat_process(lcoh, facility_id):
  h2_dem = get_plant_demand(facility_id)*365 #kg/year
  heat_dem = get_heat_demand(facility_id) #MJ/year
  # NG equivalent demand
  ng_dem = heat_dem*CONV_MJ_TO_MMBTU #MMBtu/year
  # H2 Furnace CAPEX
  gf_crf = WACC / (1 - (1/(1+WACC)**GFLT) ) 
  heat_power_dem = heat_dem/(CONV_MWh_to_MJ*365*24) #MWth
  capex_h2_bf = heat_power_dem*GFCAPEX*gf_crf #$/year
  # BE
  be_ng = (capex_h2_bf + lcoh*h2_dem)/ng_dem #$/MMBtu
  return be_ng

  

In [80]:
for facility_id in plant_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    res_be = compute_res_be_heat_process(lcoh, facility_id)
    res_be_df.loc[facility_id, res] = res_be


In [81]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_heat_process.csv', header=True, index=False)

# Ammonia

In [82]:
auxNucNH3CAPEX = 64511550 #$/year
auxNucNH3LT = 20 # years

ngNH3NGConsRate = 30.82# MMBtu/tNH3
ngNH3ElecCons = 0.061 # MWh/tNH3

In [83]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [84]:
ammonia_df = pd.read_excel('./h2_demand_ammonia_us_2022.xlsx', sheet_name='processed')
nh3_plant_ids = list(ammonia_df['id'])

In [85]:
res_be_df= pd.DataFrame(columns=['id', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['id'] = ammonia_df['id']
res_be_df.set_index('id', inplace=True)

In [86]:
from opt_deployment_ammonia import get_ammonia_plant_demand
def compute_res_be_ammonia(lcoh, lcoe, plant_id):
  ammonia_capacity, h2_dem_kg_per_day, elec_dem_MWe, state = get_ammonia_plant_demand(plant_id)#kg/year
  # NG equivalent demand
  ng_dem = ammonia_capacity*ngNH3NGConsRate #MMBtu/year
  # CAPEX costs for conversion to renewable energy
  crf = WACC / (1 - (1/(1+WACC)**auxNucNH3LT) ) 
  costs_conversion =auxNucNH3CAPEX*crf
  # Electricity costs
  costs_electricity_res = elec_dem_MWe*365*24*lcoe
  costs_electricity_ng = ammonia_capacity*ngNH3ElecCons*lcoe
  # Hydrogen costs
  costs_h2 = h2_dem_kg_per_day*365*lcoh
  # BE
  res_be = (costs_h2+costs_electricity_res+costs_conversion - costs_electricity_ng)/ng_dem
  return res_be

In [87]:
for plant_id in nh3_plant_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be_ammonia(lcoh, lcoe, plant_id)
    res_be_df.loc[plant_id, res] = res_be
res_be_df

,Grid PEM,Wind unsubsidized,Wind subsidized,Solar PEC,Solar PV-E,SMR 89% CCUS
id,,,,,,
CF_Do,86.431665,108.789788,92.391074,122.956082,126.218963,31.768321
CF_Po,86.562872,108.920995,92.522282,123.087289,126.35017,31.899528
CF_Ve,86.571299,108.929422,92.530708,123.095716,126.358597,31.907954
Ko_En,86.587529,108.945653,92.546939,123.111947,126.374828,31.924185
Io_We,86.610485,108.968608,92.569895,123.134902,126.397783,31.947141
Dy_Wa,86.643978,109.002102,92.603388,123.168396,126.431277,31.980634
Nu_Au,86.656171,109.014294,92.615581,123.180588,126.443469,31.992827
Nu_Li,86.671547,109.02967,92.630956,123.195964,126.458845,32.008203
Ya_Fr,86.671547,109.02967,92.630956,123.195964,126.458845,32.008203


In [88]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_ammonia.csv', header=True, index=False)

# Steel

In [99]:
lco_df = pd.read_excel('./input_data/lcoh_lcoe_res.xlsx', index_col=0)
res_list = list(lco_df.index)

In [100]:
steel_df = pd.read_excel('./h2_demand_bfbof_steel_us_2022.xlsx', sheet_name='processed')
steel_ids = list(steel_df['Plant'])
res_be_df= pd.DataFrame(columns=['id', 'Grid PEM', 'Wind unsubsidized', 'Wind subsidized', 'Solar PEC','Solar PV-E', 'SMR 89% CCUS'])
res_be_df['id'] = steel_df['Plant']
res_be_df.set_index('id', inplace=True)

In [101]:
import utils

dri_co2_intensity = 40 # kgCO2/ton-DRI
shaft_CAPEX = 250 # $/tDRI/year
eaf_CAPEX = 160 # $/tsteel/year
eaf_OM  = 24.89 # $/tsteel (EAF and casting)
iron_ore_cost = 100 #$/t_ironore # $/tironore
ratio_ironore_DRI = 1.391 # tironore/tDRI
bfbof_iron_cons = 1.226 #t_ironore/t_steel
om_bfbof = 178.12 #$/t_steel

from opt_deployment_steel import get_steel_plant_demand
def compute_res_be_steel(lcoh, lcoe, id):
  steel_capacity_ton_per_year, h2_dem_kg_per_day, elec_dem_MWe = get_steel_plant_demand(id)#kg/year
  # Coal equivalent demand
  coal_dem = steel_capacity_ton_per_year*utils.coal_to_steel_ratio_bau*utils.coal_heat_content #MMBtu/year
  # CAPEX costs for conversion to renewable energy
  crf = WACC / (1 - (1/(1+WACC)**20) ) 
  costs_conversion= steel_capacity_ton_per_year*(eaf_CAPEX*crf*(1-utils.ITC_H2) \
                                                 + shaft_CAPEX*(1-utils.ITC_H2)*crf/utils.steel_to_dri_ratio)
  # Electricity costs
  costs_electricity_res = elec_dem_MWe*365*24*lcoe
  # Hydrogen costs
  costs_h2 = h2_dem_kg_per_day*365*lcoh
  # O&M costs for EAF
  costs_om_eaf = eaf_OM*steel_capacity_ton_per_year
  # O&M costs for BAU case
  bau_om_costs = iron_ore_cost*bfbof_iron_cons*steel_capacity_ton_per_year +\
     om_bfbof*steel_capacity_ton_per_year
  # BE
  res_be = (costs_h2+costs_electricity_res+costs_conversion + costs_om_eaf - bau_om_costs)/coal_dem
  return res_be


In [102]:
for plant_id in steel_ids:
  for res in res_list:
    lcoh = lco_df.loc[res, 'LCOH']
    lcoe = lco_df.loc[res, 'LCOE']
    res_be = compute_res_be_steel(lcoh, lcoe, plant_id)
    res_be_df.loc[plant_id, res] = res_be
res_be_df

,Grid PEM,Wind unsubsidized,Wind subsidized,Solar PEC,Solar PV-E,SMR 89% CCUS
id,,,,,,
U.S. Steel Granite City Works,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Riverdale steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Burns Harbor steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Indiana Harbor steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
U.S. Steel Gary Works,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Dearborn steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Cleveland steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
Cleveland-Cliffs Middletown steel plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819
U.S. Steel Edgar Thomson Plant,11.456618,18.723667,12.976929,23.283701,24.060415,-5.342819


In [103]:
res_be_df = res_be_df.mean().to_frame(name='Breakeven price ($/MMBtu)')
res_be_df.reset_index(inplace=True)
res_be_df.rename(columns={'index':'RES'},inplace=True )
res_be_df.to_csv('./results/res_be_steel.csv', header=True, index=False)